In [1]:
path = "drive/My Drive/Colab Notebooks/466-proj2/"
f = open(path + 'readme.txt', 'r')
print(f.read())
f.close()

Digital Democracy Data Sets (for Project 2 groups)
The ordering for the tsv as well as what each represents (committee_utterance file)

1)vid                   - Video Id
2)fileid                - YouTube Id
3)cid                   - Committee Id
4)c_name                - Committee Name
5)c_house               - Committee House
6)hid                   - Hearing Id
7)position              - Order of bills discussed in the hearing/video order
8)pid                   - Person Id
9)diarization_id        - Id of a speaker given perfect diarization
10)last                 - Speaker last name
11)first                - Speaker first name
12)start                - Start time of utterance
13)end                  - End time of utterance
14)utterance_order      - Order of the utterance in a given hearing
15)text                 - Text of the utterance

DDDataSet_3.tsv

Use this data set for your baseline machine learning analysis of speaker identification. There are about 10,000 randomly selected 

In [2]:
FILE = "committee_utterances.tsv"

import pandas as pd

df = pd.read_csv(path + FILE, sep='\t')
df

,vid,fileid,cid,c_name,c_house,hid,position,pid,diarization_id,last,first,start,end,utterance_order,text
0,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,6,24,1,"Go ahead and call the hearing to order, our sp..."
1,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,24,41,2,But also to the communities and it's had other...
2,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,41,56,3,limited water resources are all challenges tha...
3,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,56,71,4,"And we've invited expert witnesses, who'll dis..."
4,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,71,88,5,Drought is certainly a critical issue for Cali...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30858,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1485,1494,555,"Current vote count is 9-3, and that bill is ou..."
30859,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1494,1498,556,Anderson? Mitchell? (aye) Moorlach?
30860,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1504,1512,557,Current vote count is 9-2. That bill is out. I...
30861,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1512,1521,558,Morrell? (no) Anderson? Mitchell? (aye) 9-3.


In [3]:
#Select a random 25% (1/4) of the content
import random
from pprint import pprint

records = list(df.text)
number_selected = len(records) // 100  #TODO CHANGE THIS VALUE TO 4
selected_records = random.sample(records, number_selected) #get random sample of number_selected records without replacement

utterances = [record for record in selected_records] #get all the utterance fields only (column #15) for clustering
print("total number of points:",len(utterances))

total number of points: 308


In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.corpus import wordnet 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
punctuation = "-!?;:\"\'.,"

def getPOS(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def getFeatures(text):
    features = {}
    tokens = nltk.word_tokenize(text.lower())
    tot_cnt = len(tokens)
    for word in tokens:        
        if word not in stop_words and word not in punctuation:
            w = wordnet_lemmatizer.lemmatize(word, getPOS(word))
            if w not in features:
                features[w] = 1/float(tot_cnt)
            else:
                features[w] += 1/float(tot_cnt)
    return features


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
import numpy as np
import random, sys
from collections import Counter

max_t = 2   # max itterations (ignores threshold) (-1 to unset)
k = 2
M = []
M.append([0 for i in range(0, k)])

def initCentroids(vec_data, k):
    centroidID = np.random.permutation(len(vec_data))[:k]
    return [id for id in centroidID]

def calculateDistance(v_dict, u_dict):
    v = list(v_dict.values())
    u = list(u_dict.values())
    total = 0
    for i in range(0, len(v)):
        total += ((v[i] - u[i]) ** 2)
    return np.sqrt(total)


# takes a vector dictionary and a set of all the keys
# returns a new vector dictionary with all the keys
def reshapeVectDict(vect_dict, keys):
    old_shape = len(vect_dict)
    for key in keys:
        if key not in vect_dict:
            vect_dict[key] = 0.0
    return vect_dict

# For a given point, find the closest centroid from the list, return centroid's index
def closestCentroid(point, centroids, data):
    # Get all the keys (words) in the point and centroids
    # This will be used to transform them to the same dimension so we can do math
    all_keys = []
    all_keys.extend(point)        
    for centroid in centroids:        
        all_keys.extend(data[centroid].keys())
    all_keys = set(all_keys)

    closestCentroid = 0
    minDist = sys.maxsize
    point_reshape = reshapeVectDict(point, all_keys)

    for cID in centroids:
        centroid = data[cID]
        centroid_reshape = reshapeVectDict(centroid, all_keys)
        dist = calculateDistance(point_reshape, centroid_reshape)

        if (dist < minDist):
            minDist = dist          
            closestCentroid = cID

    return closestCentroid


def k_means(vec_data, k, e):
    t = 0
    M[t] = initCentroids(vec_data, k)

    if max_t >= 0:        
        while (t < max_t):
            print(t)
            t += 1
            M.append([0 for i in range(0, k)])    # M needs t rows
            C = []
            for i in range(0, k):
                C.append([])
            # Centroid assignment
            for point in vec_data:
                clusterID = closestCentroid(point, M[t-1], vec_data)
                index = M[t-1].index(clusterID)
                C[index].append(point)
            # Centroid update
            # for i in range(0, k):
            #     M[t][i] = 

In [6]:
data = selected_records
vec_data = [getFeatures(text) for text in data]
print(len(vec_data))

308


In [11]:
import time

startTime = time.time()
endTime = time.time()

k_means(vec_data, 2, 1)

print(round(endTime - startTime,3),'seconds')

0
1
0.0 seconds


In [12]:
# M holds the centroid IDs
M

[[67, 44], [0, 0], [0, 0]]